In [4]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats.mstats import winsorize
from google.colab import drive, files
import io

# Cargar datos y dividir dataset

In [5]:
from tensorflow.random import set_seed
set_seed(234730)

In [6]:
from google.colab import drive
drive.mount('/content/drive')
train = pd.read_csv("/content/drive/MyDrive/train.csv")
test = pd.read_csv("/content/drive/MyDrive/test_without_label.csv")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
train['id'] = train['id'].astype(str)
train['tweet'] = train['tweet'].astype(str)
train['label'] = train['label'].astype(str)

In [8]:
train.drop(['id'], axis=1)
train.set_index("id", inplace = True)

In [9]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
# parseo de category
train[['label']] = train[['label']].apply(lambda col: label_encoder.fit_transform(col))

# Stopwords, lemmatize, punctuacion
Hay muchas palabras que estan repetidas como "august", "august,", "august.", "august:", etc. Para resolver esto vamos a sacar la punctuación antes de hacer el lemmatizer. Por ahora vamos a dejar algunos símbolos como los "#", quizas los hastags pueden tener información importante.

Como en el órden de ejecución del CountVectorizer primero se hace el lemmatizing,  no podemos ponerlo en las stop words. Por lo tanto lo vamos a hacer en el mismo método de lemmatizing.

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import WordNetLemmatizer
import nltk

nltk.download('wordnet')
Lemmatizer = WordNetLemmatizer()
#analyzer = CountVectorizer().build_analyzer()

from zipfile import ZipFile
file_loc = '/root/nltk_data/corpora/wordnet.zip'
with ZipFile(file_loc, 'r') as z:
  z.extractall('/root/nltk_data/corpora/')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [11]:
All_punct = '''!()-–—•…[]{};:'"“”‘’\,<>./?%^&*_~'''
#Separators ='₹$€'
#CountVectorizer con stopwords de Natural Language Toolkit
nltk.download('stopwords')
emojis = "👐🇦💺🔑💚ℹ️↗️⏺️🔽▪️◾▶️☎️☑☺⚖️⛪✅✈✉️✔️❄️❌❌❗❤️➡➡️�🆕￼🆙🌍🌐🏠🏪🏥🏫👇👉👉🏾👨🧑🧒👩⚕️💉💊📃📈📉📍📏📢📨📺🔰🔴🔹️🕌🕐🕖🕘🕗😷🙏🚶♀️🛡️🟢🤔😖😱😎😢😞😧😣😅😂🙄🤱🏿🥼🧤🧪📱🙊📣🇿🇲🌎🌎🌏🍔🎧💦🐍🐕👀👍👏👦👧👦🙌🚍🚎🚓🚨🚫🟠🟡🥡🥫🦟🧑🧔🧫🧬"


def is_Not_Link(word):
  return word[0:4] != "http"

def remove_Punctuation(doc_split):
  for i in range(len(doc_split)):
    word = doc_split[i];
    for elements in word:
      if elements in All_punct:
        word = word.replace(elements, "")
        doc_split[i] = word     
  return doc_split

def remove_Stopwords(doc_split):
  doc_with_Stopwords = doc_split.copy();
  for i in range(len(doc_split)):
    word = doc_split[i];
    if word in nltk.corpus.stopwords.words('english'):
      doc_with_Stopwords.remove(word);

  return doc_with_Stopwords

def separate_Emojis(doc_split):
  doc_with_Emojis = doc_split.copy();
  for i in range(len(doc_split)):
    word = doc_split[i];
    for elements in word:
        if elements in emojis:
          doc_with_Emojis.append(elements)
          word = word.replace(elements, "")
          doc_with_Emojis[i] = word
  return doc_with_Emojis

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [12]:
Hindi = ['आँकड़ा', 'इस', 'कर', 'करने', 'के', 'को', 'ख़त्म', 'छोटी', 'जड़', 'ज़रूरी', 'जाएगा।', 'देश', 'पार', 'बहुत', 'में', 'लिए', 'सावधानियाँ', 'सी', 'से', 'हफ़्ते', 'हमारे', 'है']
Chinese = ['監獄絕食抗議', '被監視居住']

In [13]:
def remove_Foreign_Languages(doc_split):
  doc_without_Languages = doc_split.copy();
  for i in range(len(doc_split)):
    word = doc_split[i];
    if (word in Hindi) or (word in Chinese):
      doc_without_Languages.remove(word);

  return doc_without_Languages

In [14]:
Unicode = ['\u200d','\u200b', '\u2063', '\u2066', '\u2069']

def remove_Unicode(doc_split):
  for i in range(len(doc_split)):
    word = doc_split[i];
    for j in range(len(word)):
      char = word[j];
      if char in Unicode:
        doc_split[i] = doc_split[i].replace(char, "")
  return doc_split

In [15]:
text = "\u200dhello world\u200d this is amazinjajaj\u200dhola ogla"
#text.replace('\u200d', "")
if(text[0] == "\u200d"):
  print("true 1")
if(text[12] == "\u200d"):
  print("true 2")

true 1
true 2


In [16]:
a = remove_Unicode(text.split())
a

['hello', 'world', 'this', 'is', 'amazinjajajhola', 'ogla']

In [17]:
def remove_Empty_Spaces(doc_split):
  doc_split =  [x for x in doc_split if x]
  return doc_split

In [18]:
split = ["", "hola", "", "mundo", ""]
split =  [x for x in split if x]
print(split)

['hola', 'mundo']


In [19]:
def remove_Links(doc_split):
  doc_without_Links = doc_split.copy();
  for i in range(len(doc_split)):
    word = doc_split[i];
    while (word[0] in All_punct) and (len(word) > 1):
      word = word[1:len(word)]
    if not(is_Not_Link(word)):
      doc_without_Links.remove(word);
  return doc_without_Links;

In [20]:
ciudadesNigeria = ["Lagos", "River", "FCT", "Abia", "Borno", "Oyo", "Rivers", "Edo", "Gombe", "Ogun", "Plateau", "Delta", "Bauchi", "Kaduna", "Benue", "Ondo", "Kwara", "Nasarawa", "Enugu", "Sokoto", "Niger", "Kebbi", "Yobe", "Kano"]

import re
def separate_Numbers(doc_split):
  for i in range(len(doc_split)):
    word = doc_split[i];
    for cities in ciudadesNigeria:
      if cities in word:
        match = re.match(r"([a-z]+)([0-9]+)", word, re.I)
        if match:
          doc_split[i] = match.groups()[0]
          for i in range(len(match.groups())):
            if i != 0:
              doc_split.append(match.groups()[i])
  return doc_split

In [56]:
def separate_Arrobas(doc_split):
  for i in range(len(doc_split)):
    word = doc_split[i];
    if word[0] == "@": 
      break
    wordSeparated = word.split("@")
    doc_split[i] = wordSeparated[0]
    aux = 1
    while aux < len(wordSeparated):
      wordWithArroba = "@"
      wordWithArroba += wordSeparated[aux]
      doc_split.append(wordWithArroba)
      aux += 1
    
  return doc_split

In [57]:
def new_analyzer(doc):
  doc_split = doc.split();
  doc_split = remove_Links(doc_split);
  doc_split = remove_Punctuation(doc_split);
  doc_split = separate_Emojis(doc_split);
  doc_split = remove_Stopwords(doc_split);
  doc_split = remove_Unicode(doc_split);
  doc_split = remove_Foreign_Languages(doc_split);
  doc_split = remove_Empty_Spaces(doc_split);
  doc_split = separate_Numbers(doc_split);
  if ("@" in doc):
    doc_split = separate_Arrobas(doc_split);
  return (Lemmatizer.lemmatize(w.lower()) for w in doc_split);

nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [23]:
All_punct[5]

'—'

# Ingenieria de atributos

In [58]:
letters = "abcdefghijklmnñopqrstuvwxyz"
def contarLetras(row):
  count = 0;
  for letter in row['tweet']:
    if letter in letters:
      count = count +1;
  return count

def contarCovid(row):
  count = 0;
  if "covid" in row['tweet']:
    return 1
  return 0

def contarHashtags(row):
  count = 0;
  for letter in row['tweet']:
    if letter == "#":
      count = count +1;
  return count * 10

emojis = "👐🇦💺🔑💚ℹ️↗️⏺️🔽▪️◾▶️☎️☑☺⚖️⛪✅✈✉️✔️❄️❌❌❗❤️➡➡️�🆕🆙🌍🌐🏠🏪🏥🏫👇👉👉🏾👨🧑🧒👩⚕️💉💊📃📈📉📍📏📢📨📺🔰🔴🔹️🕌🕐🕖🕘🕗😷🙏🚶♀️🛡️🟢🤔😖😱😎😢😞😧😣😅😂🙄🤱🏿🥼🧤🧪📱🙊📣🇿🇲🌎🌎🌏🍔🎧💦🐍🐕👀👍👏👦👧👦🙌🚍🚎🚓🚨🚫🟠🟡🥡🥫🦟🧑🧔🧫🧬"
def contarEmojis(row):
  count = 0;
  for letter in row['tweet']:
    if letter in emojis:
      count = count + 1
  return count

In [59]:
train['largoDocumento'] = train.tweet.str.len()
train['CantidadPalabras'] = train.apply(lambda row : len(row['tweet'].split()), axis=1)
train['cantidad de letras'] = train.apply(lambda row : contarLetras(row), axis=1)
train['cantidad de covid'] = train.apply(lambda row : contarLetras(row), axis=1)
train['cantidad de hashtag'] = train.apply(lambda row : contarHashtags(row), axis=1)
train['cantidad de emojis'] = train.apply(lambda row : contarEmojis(row), axis=1)

# Train / Validation
Al final de la ingeniería dividimoms en train y validation

In [60]:
from sklearn.model_selection import train_test_split

X_train, X_validation, Y_train, Y_validation = train_test_split(
    train.drop('label', axis=1),
    train['label'],
    test_size=(1.0/3), random_state=42)

## Fit

In [61]:
TF_IDF_Vectorizer = TfidfVectorizer(analyzer=new_analyzer, max_features=15000)
bag_of_words = TF_IDF_Vectorizer.fit(X_train["tweet"])
bag_of_words_fitted = bag_of_words
print(bag_of_words.get_feature_names())

['#', '##3600mg', '#1', '#2', '#2020presidentialelection', '#21dayschallenge', '#21dayslockdown', '#251', '#46', '#a', '#ableg', '#achristmascarol', '#acog2020', '#active', '#activecases', '#actnow', '#acuteflaccidmyelitis', '#afaracare', '#afoolat60', '#ahmedabad', '#airline', '#aliens', '#allergies', '#alllivesmatter', '#alzheimers', '#amazon', '#america', '#amitabhbachchan', '#andhrapradesh', '#anteater', '#antibiotics', '#aoc', '#apocalypse', '#appeal', '#area', '#arrestfauci', '#arrestgates', '#arunachalpradesh', '#arvindkejriwal', '#asco20', '#ashas', '#asians', '#asymptomatic', '#athens', '#auspol', '#autism', '#autoimmune', '#avifavir', '#aworldindisorder', '#azimpremji', '#backtoschool', '#bacteria', '#bailout', '#bankerhaipolicenahi', '#baptism', '#barackobama', '#baseball', '#basketball', '#be', '#beatncds', '#beer', '#bengali', '#berniesanders', '#bharatbiotech', '#bigpharmapuppets', '#bihar', '#binaxnow', '#bioweapon', '#boycottchineseproduct', '#boycottmadeinchina', '#bre

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


## Transform

In [62]:
bag_of_words = TF_IDF_Vectorizer.transform(X_train["tweet"])
bag_of_words = pd.DataFrame(bag_of_words.toarray(), index = X_train.index, columns = TF_IDF_Vectorizer.get_feature_names())

In [63]:
bag_of_words.shape

(5706, 13855)

## Añadir atributos al nuevo dataframe de TFIDF

In [64]:
bag_of_words['largoDocumento'] = X_train['largoDocumento']
bag_of_words['CantidadPalabras'] = X_train['CantidadPalabras']
bag_of_words['cantidad de letras'] = X_train['cantidad de letras']
bag_of_words['cantidad de covid']  = X_train['cantidad de covid'] 
bag_of_words['cantidad de hashtag'] = X_train['cantidad de hashtag']
bag_of_words['cantidad de emojis'] = X_train['cantidad de emojis']

In [65]:
bag_of_words.shape[0]

5706

## Add Weights

In [66]:
#for i in range(5706):
#  if bag_of_words['trump'][i] > 0:
#    print(i)

In [67]:
#position = bag_of_words_fitted.vocabulary_['trump']
#position

In [68]:
#for i in range(bag_of_words.shape[0]):
#  bag_of_words['trump'][i] = bag_of_words['trump'][i] * 2

### Important columns


In [69]:
columns = ['#trump2020', 'trump', '@realdonaldtrump']

In [70]:
#for i in range(len(columns)):
#  column = columns[i];
#  for row in range(bag_of_words.shape[0]):
#    bag_of_words[column][row] = bag_of_words[column][row] * 1.2

### All Hashtags


#### List

In [71]:
hashtags = ['#', '##3600mg', '#1', '#2', '#2020presidentialelection', '#21dayschallenge', '#21dayslockdown', '#251', '#46', '#a', '#ableg', '#achristmascarol', '#acog2020', '#active', '#activecases', '#actnow', '#acuteflaccidmyelitis', '#afaracare', '#afoolat60', '#ahmedabad', '#airline', '#aliens', '#allergies', '#alllivesmatter', '#alzheimers', '#amazon', '#america', '#amitabhbachchan', '#andhrapradesh', '#anteater', '#antibiotics', '#aoc', '#apocalypse', '#appeal', '#area', '#arrestfauci', '#arrestgates', '#arunachalpradesh', '#arvindkejriwal', '#asco20', '#ashas', '#asians', '#asymptomatic', '#athens', '#auspol', '#autism', '#autoimmune', '#avifavir', '#aworldindisorder', '#azimpremji', '#backtoschool', '#bacteria', '#bailout', '#bankerhaipolicenahi', '#baptism', '#barackobama', '#baseball', '#basketball', '#beatncds', '#beer', '#bengali', '#berniesanders', '#be…', '#bharatbiotech', '#bigpharmapuppets', '#bihar', '#binaxnow', '#bioweapon', '#boycottchineseproduct', '#boycottmadeinchina', '#breakfast', '#breakin', '#breaking', '#breakinghot', '#breastfeed', '#bronxzoo', '#brooklyn', '#buddhism', '#bulls', '#burgerking', '#burglars', '#burrgate', '#business', '#c19', '#california', '#californ…', '#canada', '#cancelcacsexams', '#cancer', '#cardiotwitter', '#caregivers', '#carona', '#cases', '#cashewchicken', '#ccc19', '#ccp', '#ccpvirus', '#cdc', '#cdcgrandrounds', '#cdctravelnotice', '#cdnpoli', '#cdntv', '#celebrities', '#cerb', '#chandigarh', '#charleslieber', '#cheating', '#chennai', '#chenqiushi', '#children', '#china', '#china@drtedros', '#chinacoronavirus', '#chinaindiafaceoff', '#chinese', '#chinesecommunistparty', '#chinesevirus', '#chloroquine', '#christmas', '#cia', '#classof2020', '#clinicaltrials', '#clinicians', '#codecheck', '#coevolution', '#coexistence', '#colorado', '#computers', '#connecticut', '#conspiracy', '#conspiracytheories', '#contacttracing', '#containmentzones', '#convalescent', '#cornavirusfacts', '#corona', '#coronabeer', '#coronacheck', '#coronainpakistan', '#coronaoutbreak', '#coronaupdate', '#coronaupdates', '#coronaupdatesindia', '#coronaupdatesinindia', '#coronavacccine', '#coronavaccine', '#coronavirus', '#coronaviruscanada', '#coronavirusfacts', '#coronavirusincontext', '#coronavirusindia', '#coronaviruslockdown', '#coronavirusoutbreak', '#coronaviruspandemic', '#coronavirustruth', '#coronavirusupdate', '#coronavirusupdates', '#coronavirususa', '#coronavirus…', '#coronawarriors', '#coronawatch', '#coronil', '#corononavirus', '#coronovirus', '#corruption', '#covax', '#covaxin', '#covaxine', '#covid', '#covid19', '#covid19@drtedros', '#covid19affected', '#covid19asserts', '#covid19associated', '#covid19india', '#covid19like', '#covid19my', '#covid19nc', '#covid19nigeria', '#covid19nz', '#covid19pakistan', '#covid19pandemic', '#covid19related', '#covid19science', '#covid19toronto', '#covid19updates', '#covid19vaccine', '#covid19…', '#covid1…', '#covid2019', '#covidactnow', '#covidcanada', '#covidindiaseva', '#covidlockdown', '#covidpandemic', '#covidstopswithme', '#covidtesting', '#covidupdate', '#covidupdates', '#covidview', '#covidwarriors', '#covidwarriortips', '#covid…', '#covidー19', '#covidー19…', '#cowurinesteambar', '#cpd', '#criminals', '#crook', '#cruise', '#cruiseships', '#cure', '#curfewinindia', '#dallascowboys', '#data', '#datoscoronavirus', '#day2lockdown', '#dearicaitake', '#death', '#defense', '#delhi', '#delhifightscorona', '#dementia', '#democrats', '#denmark', '#dexamethasone', '#dialysis', '#diapers', '#dictatorship', '#disease', '#diseasedetective', '#disinfectant', '#disney', '#dna', '#domesticviolence', '#donaldtrump', '#doortodoordeworming', '#doublingrate', '#doyourpart', '#dranthonyfauci', '#drugs', '#duckdynasty', '#dyk', '#ears', '#easyoff', '#ebolavirus', '#echinacea', '#economy', '#eha2020', '#eidaladha', '#elonmusk', '#endtb', '#epass', '#epidemic', '#erscongress', '#esccongress', '#essentialworker', '#eular2020', '#evin', '#evinindia', '#exercise', '#facebook', '#facecover', '#facemask', '#facemasks', '#factcheck', '#factchecking', '#factsmatter', '#fake', '#fakenews', '#fakenewsalert', '#families', '#family', '#famine', '#fangbin', '#faq', '#fatality', '#fatalityrate', '#fauci', '#favipiravir', '#fbi', '#fda', '#febreeze', '#feelbetter', '#feeling', '#fidelcastro', '#fightagainstcoronavirus', '#fightflu', '#firefighters', '#fitness', '#flattenthecurve', '#flight', '#florida', '#flu', '#foghornleghorn', '#food', '#foodsafety', '#foodshortages', '#footballamerican', '#ford', '#fourthofjuly', '#foxnews', '#freedom', '#friendship', '#fullyimmunizeeverychild', '#fun', '#gandhinagar', '#gaumutrsteaming', '#gautambuddhnagar', '#germs', '#ghaziabad', '#globalwarming', '#god', '#godelhiites', '#goingbust', '#golf', '#government', '#governor', '#goyabeans', '#grafitti', '#great', '#greece🇬🇷', '#groupies', '#gujarat', '#gujaratcoronaupdate', '#gurugram', '#guyana', '#hairstylist', '#hallmark', '#hamburger', '#hands', '#handwashing', '#hantavirus', '#harrymaguire', '#harvarduniversity', '#haryana', '#hcp', '#hcps', '#hcq', '#health', '#healthcare', '#healthequity', '#healthforall', '#healthoverexams', '#healthworkers', '#healthy', '#healthylife', '#heartattacks', '#heat', '#heidiklum', '#heneryhawk', '#himachalpradesh', '#hit2equity', '#hitsolutions', '#hiv', '#hoarding', '#hockey', '#hollywood', '#homeisolation', '#homelessness', '#homemade', '#homeopathic', '#homequarantine', '#hongkongs', '#howwehospitalist', '#huaweiwatchgt2', '#hurricane', '#hurricanelaura', '#hydroxychloroquine', '#hypertension', '#ibs', '#icairespondnow', '#icmrfightscovid19', '#icu', '#igot', '#illegalimmigrants', '#immunesystem', '#immunity', '#immunitybooster', '#immunizationforall', '#impeachment', '#importantinformation', '#incontext', '#india', '#indiafightscirona', '#indiafightscorona', '#indiafightscoronavirus', '#indiafightscovid19', '#indiawillwin', '#india…', '#indigo', '#infections', '#inflammation', '#ingenuity', '#institutionalquarantine', '#introvert', '#ireland', '#irony', '#istandwithmashpee', '#italy', '#iud', '#jamalkhashoggi', '#japan', '#jaredkushner', '#jarkhand', '#jennymccarthy', '#jharkhand', '#jlo', '#jobs', '#johntravolta', '#josephbiden', '#journalist', '#kag', '#kano', '#karakaxlist', '#karnataka', '#karnatakas', '#kayburley', '#keeppushing', '#keithholdskeytovirusvaccine', '#kerala', '#kidneytransplant', '#kids', '#killingvirus', '#kishanganj', '#kitchen', '#knowthefacts', '#kolkataairport', '#laborday', '#lachupacabra', '#lakshadweep', '#language', '#laura', '#lcsm', '#lego', '#lgbtquarantine', '#lies', '#life', '#list', '#livelife', '#lockdown', '#lockdown2', '#lockdown21', '#lockdown4', '#lockdownextension', '#lockdownnow', '#lockdownquery', '#london', '#louisiana', '#lucknow', '#lysol', '#madhyapradesh', '#maga', '#maharashtra', '#maharastra', '#mailinvoting', '#mainbhinewschecker', '#majorleaguebaseball', '#makhi', '#malaria', '#malnutrition', '#malpractice', '#manafort', '#manipur', '#marchmadness', '#marijuana', '#mascot', '#mask', '#maskingforafriend', '#maskonnaija', '#masks', '#mauritius', '#mayans', '#meat', '#media', '#medical', '#medicaldoctor', '#medicines', '#medtwitter', '#mentalhealth', '#mergecaexamsjulyandnov', '#metformin', '#metro', '#metrobackontrack', '#mexico', '#michigan', '#minnesota', '#miraclesoftrueworship', '#missiles', '#missionkarmayog…', '#mitchmcconnell', '#mizoram', '#modicovidaddress', '#modigovtfailed', '#modijipostponejeeneet', '#money', '#moneylaundering', '#morningmotivation', '#mother', '#mouthwash', '#movies', '#mrna', '#msvirtual2020', '#mumbai', '#muradabad', '#n95', '#nagpur', '#nails', '#nancypelosi', '#napoleon', '#nashville', '#nba', '#ncdcinthenews', '#ncdclabnetwork', '#ncdcrrt', '#ncdcteam', '#ncdhhs', '#ncpol', '#nepotism', '#newhampshire', '#news', '#newschecker', '#newsinphoto', '#newssatire', '#newyork', '#newyorkcity', '#newyorktimes', '#newzealand', '#nhl', '#nigeria', '#nigerian', '#nigerians', '#noahsark', '#noida', '#norway', '#notdying4wallstreet', '#notmypresident', '#novel', '#nursinghome', '#nyc', '#obamagate', '#obesewomen', '#oct1stprotest', '#odisha', '#odishafightscorona', '#office', '#officialsecretsact', '#ohio', '#olympicgames', '#online', '#onpoli', '#opthalmology', '#orlando', '#oslo', '#panamacanal', '#pandas', '#pandemic', '#panic', '#panicbuying', '#parasites', '#parent', '#parents', '#patientzero', '#pcrtest', '#peepingtom', '#philrobertson', '#physicaldistance', '#physique', '#pibfactcheck', '#placebopills', '#plandemic', '#plasmatherapy', '#plastic', '#pmqs', '#pmsby', '#poker', '#politicalcorrectness', '#politicians', '#politics', '#pope', '#potsales', '#potus', '#ppe', '#prep', '#prescriptions', '#preventcovid19', '#prevention', '#prioritycovidresearch', '#privacy', '#propaganda', '#property', '#prostatecancer', '#ptfcovid19', '#pti', '#pui', '#qanda', '#quarantine', '#queens', '#rackets', '#rajahthan', '#rajasthan', '#rapidantigentest', '#rc70afro', '#recession', '#recoveryrate', '#religion', '#remdesivir', '#remedy', '#rentfreezenow', '#reopeningsafely', '#reopensafely', '#republicans', '#research', '#revolutionnow', '#rheaarrested', '#rice', '#richardnixon', '#ridge', '#rnc2020', '#roadtrips', '#rockandrollvaccine', '#rollingstones', '#rollingstonevaccineforcv', '#rpharmcoronavir', '#rtpcr', '#russia', '#russiaeverywhere', '#russianvaccine', '#russie', '#sacrifice', '#safe', '#safetyfirst', '#sally', '#sam2020', '#santa', '#sarscov2', '#satire', '#saturdaynightfever', '#saudiarabia', '#school', '#schoolsreopening', '#schwabeindia', '#science', '#seanhannity', '#selfcare', '#selfisolation', '#sens', '#sepsis', '#sex', '#sexyteens', '#share', '#sheep', '#sickness', '#simoncowell', '#sitamarhi', '#slowthespread', '#smallpox', '#smartnews', '#socialdistance', '#socialdistancing', '#sofi2020', '#sofiavergara', '#southcarolina', '#southdelhi', '#spain', '#spokesman', '#sponsors', '#sport', '#sports', '#springst', '#sputnikv', '#stayalert', '#stayathome', '#stayathomesavelives', '#stayhealthy', '#stayhome', '#stayhomestaysafe', '#staysafe', '#staystrong', '#std', '#stds', '#stevenseagal', '#stimuluschecks', '#stop', '#stopthespread', '#stopthespreadofcorona', '#strategicreserve', '#stress', '#strokes', '#suicide', '#summer', '#sunbathing', '#swasthabharat', '#swedish', '#symptomatic', '#t1d', '#takeaction', '#takebreaks', '#takeresponsibility', '#takeresponsibilty', '#tamilnadu', '#tan', '#tb', '#teamoffivemillion', '#tech', '#telangana', '#telemedicine', '#televangelist', '#telugunews', '#teravolt', '#testing', '#tests', '#texas', '#thailand🇹🇭', '#thane', '#thankshealthheroes', '#thedevil', '#thenewnormal', '#thesun', '#thewiderwiserview', '#tigers', '#time', '#tipsadvice', '#togetheralone', '#toilet', '#toiletpaper', '#toiletrollhorders', '#tomhanks', '#travel', '#trinitybellwoodspark', '#tripura', '#trophywife', '#trump2020', '#trumpcouldhavesavedvalet', '#trumppressconf', '#trumpsjealousofobama', '#trumpuniversity', '#tsunami', '#tuberculosis', '#tuesdaylive', '#tuesdaymotivation', '#tulsatrumprally', '#tv', '#tīmarimamiriona', '#ufo', '#uighurs', '#un75', '#unemployment', '#unga', '#unitednations', '#unlock1', '#unlock2', '#unlock3', '#unlock3guidelines', '#unlock4', '#unlock4guidelines', '#update', '#uruguay', '#usa', '#uttarpradesh', '#uva', '#uvc', '#uvdisinfection', '#uvlamp', '#uvled', '#uvlight', '#uvsolution', '#vacation', '#vaccine', '#vaccines', '#vaccineswork', '#vadodara', '#vegan', '#vermont', '#vicepresidentmikepence', '#vicpol', '#viral', '#viralvideo', '#virtualconferenceafrica', '#virus', '#viruses', '#visualabstract', '#vitamind', '#vladimirputin', '#war', '#warcrimes', '#warning', '#warnings', '#washington', '#waterpistols', '#wear', '#wearafacemask', '#wearamask', '#wearamask…', '#weather', '#weneedthedenominator', '#westbengal', '#westvirginia', '#wha73', '#wha74', '#whatsapp', '#whitehouse', '#whitesupremacy', '#who', '#whoimpact', '#wildfire', '#wildfires', '#williambarr', '#williambryan', '#woodwardtapes', '#work', '#workfromhome', '#worldbreastfeedingweek', '#worldmaskweek', '#worldmosquitoday', '#worldzoonosesday', '#worried', '#wor…', '#wtd20', '#wuhan', '#wuhancoronavirus', '#wuhaninstituteofvirologylaboratory', '#wuhanviruspandemic', '#xijinping', '#yavatmal', '#yourviewtv', '#zhangzhan', '#zoodirector']

#### Code

In [72]:
#for i in range(len(hashtags)):
#  print(i)
#  hashtag = hashtags[i];
#  for row in range(bag_of_words.shape[0]):
#    bag_of_words[hashtag][row] = bag_of_words[hashtag][row] * 1.5

# Red Neuronal

In [73]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Bidirectional, Conv1D
from tensorflow.keras.layers import LeakyReLU

In [74]:
#mejor densas: 20 - 50 - 40 -> 0.9432375613174492
  # 30-50-40 dio lo mismo
  # 30-60-50 --> NO
  # 20-50-30 --> NO
  # 20-30-40 --> NO - 0.940084092501752

#Activaciones
  #Leaky-Relu x2-Sigmoid --> 0.9432375613174492
  #Leaky-relu-Sigmoidx2 --> 0.9470918009810791 --> Mejor
  #Leaky-Sigmoid-relu-Sigmoid --> 0.9397337070777856
  #Sigmoid-relu-Sigmoidx2 --> 0.9442887175893483
  #relu-Leaky-Sigmoidx2 -->  0.9425367904695164
  #relux2-Sigmoidx2 --> 0.933426769446391
  #Leaky-relu-Leaky-Sigmoid --> 0.9418360196215837

In [75]:
adam_optimizer='Adam'
loss='binary_crossentropy'
metric='accuracy'
#sqrt(inputlayers * outputlayer) = sqrt(15000) = 122
#model.add(GRU(1, input_shape=(20,1)))

model = Sequential()
model.add(Dense(30, input_shape=(bag_of_words.shape[1],)))
model.add(Activation(LeakyReLU(alpha=0.3)))
model.add(Dense(50, use_bias=True))
model.add(Activation('relu'))
model.add(Dense(40, use_bias=True))
model.add(Activation('sigmoid'))
model.add(Dense(1, use_bias=True))
model.add(Dropout(0.2))
model.add(Activation('sigmoid'))
model.compile(loss=loss, optimizer=adam_optimizer, metrics=[metric])

In [76]:
X_train_array = np.asarray(bag_of_words)
Y_train_array = np.asarray(Y_train)

training = model.fit(X_train_array, Y_train_array, epochs=200, batch_size=100)

Epoch 1/200
58/58 [==============================] - 2s 14ms/step - loss: 0.6567 - accuracy: 0.6073
Epoch 2/200
58/58 [==============================] - 1s 18ms/step - loss: 0.6029 - accuracy: 0.6851
Epoch 3/200
58/58 [==============================] - 1s 13ms/step - loss: 0.5749 - accuracy: 0.6966
Epoch 4/200
58/58 [==============================] - 1s 16ms/step - loss: 0.5356 - accuracy: 0.7170
Epoch 5/200
58/58 [==============================] - 1s 18ms/step - loss: 0.5090 - accuracy: 0.7327
Epoch 6/200
58/58 [==============================] - 1s 19ms/step - loss: 0.4278 - accuracy: 0.7836
Epoch 7/200
58/58 [==============================] - 1s 15ms/step - loss: 0.3700 - accuracy: 0.8109
Epoch 8/200
58/58 [==============================] - 1s 14ms/step - loss: 0.3090 - accuracy: 0.8400
Epoch 9/200
58/58 [==============================] - 1s 13ms/step - loss: 0.2846 - accuracy: 0.8470
Epoch 10/200
58/58 [==============================] - 1s 14ms/step - loss: 0.2252 - accuracy: 0.8728

In [77]:
import tensorflow.keras as keras

model.save("basic_model")
saved_model = keras.models.load_model("basic_model")

In [78]:
X_Validation_Bag = TF_IDF_Vectorizer.transform(X_validation["tweet"])
X_Validation_Bag_df = pd.DataFrame(X_Validation_Bag.toarray(), index = X_validation.index,columns = TF_IDF_Vectorizer.get_feature_names())

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [79]:
X_Validation_Bag_df['largoDocumento'] = X_validation['largoDocumento']
X_Validation_Bag_df['CantidadPalabras'] = X_validation['CantidadPalabras']
X_Validation_Bag_df['cantidad de letras'] = X_validation['cantidad de letras']
X_Validation_Bag_df['cantidad de covid']  = X_validation['cantidad de covid']
X_Validation_Bag_df['cantidad de hashtag'] = X_validation['cantidad de hashtag']
X_Validation_Bag_df['cantidad de emojis'] = X_validation['cantidad de emojis']

In [80]:
X_Validation_Bag_df_array = np.asarray(X_Validation_Bag_df)
Y_Validation_Predict = saved_model.predict(X_Validation_Bag_df_array)

90/90 [==============================] - 0s 3ms/step


In [81]:
def transformPrediction(df, predNumber, predLabel):
  copy = df.copy()
  for i in range(len(copy)):
    if copy[i][0] > 0.5:
      predNumber.append([1])
      predLabel.append(["real"])
    else:
      predNumber.append([0])
      predLabel.append(["fake"])


In [82]:
predValidationNumbers = []
predValidationLabels = []
transformPrediction(Y_Validation_Predict, predValidationNumbers, predValidationLabels)

In [83]:
validationNumbers = []
validationLabels = []
copy = np.asarray(Y_validation).copy()
for i in range(len(copy)):
  if copy[i] > 0.5:
    validationNumbers.append([1])
    validationLabels.append(["real"])
  else:
    validationNumbers.append([0])
    validationLabels.append(["fake"])

In [84]:
np.asarray(Y_validation)

array([1, 1, 1, ..., 0, 1, 0])

#Accuracy


In [86]:
from sklearn.metrics import accuracy_score
print(accuracy_score(predValidationLabels,validationLabels))

0.9467414155571128


In [ ]:
0.9463910301331465

0.9463910301331465

In [ ]:
0.9491941135248774

0.9491941135248774

# K fold

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
import math
k = 5

def k_folds(X, y, k):
  assert(len(X) == len(y))
  folds_X = []
  folds_y = []
  initial_pos = 0

  for i in range(k):
    to_pos = min(math.ceil(len(X)/k)*(i+1), len(X))
    
    x_fold = X[initial_pos:to_pos]
    y_fold = y[initial_pos:to_pos]

    folds_X.append(x_fold)
    folds_y.append(y_fold)
    initial_pos = to_pos

  return folds_X, folds_y

In [ ]:
def evaluate(model, X_to_eval, Y_to_eval):
    Y_Prediction = model.predict(np.asarray(X_to_eval))
    numbers = []
    prediction_labels = []
    transformPrediction(Y_Prediction, numbers, prediction_labels)

    validationLabels = []
    copy = np.asarray(Y_to_eval).copy()
    for i in range(len(copy)):
      if copy[i] > 0.5:
        validationLabels.append(["real"])
      else:
        validationLabels.append(["fake"])

    return accuracy_score(validationLabels, prediction_labels)

In [ ]:
def k_fold_cross_val(folds_X, folds_y, model):
  assert(len(folds_X) == len(folds_y))
  X_to_train = []
  y_to_train = []
  X_to_eval = 0
  y_to_eval = 0
  scores = []
  for i in range(len(folds_X)):
    X_to_train = []
    y_to_train = []
    for j in range(len(folds_X)):
      if  i == j:
        X_to_eval = folds_X[i]
        y_to_eval = folds_y[i]
      else:
        X_to_train.append(folds_X[j])
        y_to_train.append(folds_y[j])
    
    X_train = np.concatenate(X_to_train)
    y_train = np.concatenate(y_to_train)
    model.fit(X_train,y_train, epochs=200, batch_size=100, validation_split=0.2)
    score = evaluate(model, X_to_eval, y_to_eval)
    scores.append(score)
  return np.mean(np.array(scores)), scores

In [ ]:
folds_X, folds_y = k_folds(np.asarray(bag_of_words), np.asarray(Y_train), 5)


In [ ]:
k_fold_result_mean, k_fold_result_arr = k_fold_cross_val(folds_X, folds_y, model)

In [ ]:
k_fold_result_mean

0.9869947275922671

In [ ]:
k_fold_result_arr

[1.0, 1.0, 1.0, 1.0, 0.9349736379613357]

# Test

In [ ]:
test['id'].astype('str')
test['tweet'].astype('str')

test_bag = TF_IDF_Vectorizer.transform(test["tweet"])
test_bag = pd.DataFrame(test_bag.toarray(), index = test.index, columns = TF_IDF_Vectorizer.get_feature_names())

test['largoDocumento'] = test.tweet.str.len()
test['CantidadPalabras'] = test.apply(lambda row : len(row['tweet'].split()), axis=1)
test['cantidad de letras'] = test.apply(lambda row : contarLetras(row), axis=1)
test['cantidad de covid'] = test.apply(lambda row : contarLetras(row), axis=1)
test['cantidad de hashtag'] = test.apply(lambda row : contarHashtags(row), axis=1)
test['cantidad de emojis'] = test.apply(lambda row : contarEmojis(row), axis=1)

test_bag['largoDocumento'] = test['largoDocumento']
test_bag['CantidadPalabras'] = test['CantidadPalabras']
test_bag['cantidad de letras'] = test['cantidad de letras']
test_bag['cantidad de covid']  = test['cantidad de covid']
test_bag['cantidad de hashtag'] = test['cantidad de hashtag']
test_bag['cantidad de emojis'] = test['cantidad de emojis']

test_bag_array = np.asarray(test_bag)
test_prediction = saved_model.predict(test_bag_array)

copy = test_prediction.copy()
predNumber = [];
predLabel = [];
for i in range(len(copy)):
  if copy[i][0] > 0.5:
    predNumber.append([1])
    predLabel.append(["real"])
  else:
    predNumber.append([0])
    predLabel.append(["fake"])

prediction = pd.DataFrame(predLabel, columns=['label'])
prediction.index += 1
prediction = prediction.to_csv('prediction.csv')

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


67/67 [==============================] - 0s 3ms/step


In [ ]:
from google.colab import files

files.download('prediction.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>